In [8]:
import csv
import pprint
data_Neighbourhoods = list(csv.DictReader(open('Neighbourhoods.csv')))
data_Neighbourhoods

[OrderedDict([('area_id', '102011028'),
              ('area_name', 'Avoca Beach - Copacabana'),
              ('land_area', '643.8'),
              ('population', '7590'),
              ('number_of_dwellings', '2325'),
              ('number_of_businesses', '738')]),
 OrderedDict([('area_id', '102011029'),
              ('area_name', 'Box Head - MacMasters Beach'),
              ('land_area', '3208.6'),
              ('population', '10986'),
              ('number_of_dwellings', '3847'),
              ('number_of_businesses', '907')]),
 OrderedDict([('area_id', '102011030'),
              ('area_name', 'Calga - Kulnura'),
              ('land_area', '76795.1'),
              ('population', '4841'),
              ('number_of_dwellings', '1575'),
              ('number_of_businesses', '1102')]),
 OrderedDict([('area_id', '102011031'),
              ('area_name', 'Erina - Green Point'),
              ('land_area', '3379.3'),
              ('population', '14237'),
              ('number_o

In [11]:
import psycopg2

def pgconnect():
   
    try: 
        conn = psycopg2.connect(host='localhost', database='assignment', user='postgres', password='6688')
        
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
    return conn

In [12]:
def pgexec( conn, sqlcmd, args, msg, silent=False ):
   """ utility function to execute some SQL statement
       can take optional arguments to fill in (dictionary)
       error and transaction handling built-in """
   retval = False
   with conn:
      with conn.cursor() as cur:
         try:
            if args is None:
               cur.execute(sqlcmd)
            else:
               cur.execute(sqlcmd, args)
            if silent == False: 
                print("success: " + msg)
            retval = True
         except Exception as e:
            if silent == False: 
                print("db error: ")
                print(e)
   return retval

# 1st: login to database
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS Neighbourhoods", None, "Reset Table Neighbourhoods")

# 2nd: ensure that the schema is in place

neighbourhoods_schema = """CREATE TABLE IF NOT EXISTS Neighbourhoods (
                         area_id VARCHAR(20) PRIMARY KEY,
                         area_name VARCHAR(150),
                         land_area FLOAT,
                         population INT,
                         number_of_dwellings INT,
                         number_of_businesses INT,
                         boundry BOX
                   )"""
pgexec (conn, neighbourhoods_schema, None, "Create Table Neighbourhoods")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
 #insert_stmt = """INSERT INTO Neighbourhoods(area_id, area_name, land_area, population, number_of_dwellings, number_of_businesses) VALUES (%(area_id)s, %(area_name)s, %(parent_area_id)s)"""
# for row in data_organisations:
 #   pgexec (conn, insert_stmt, row, "row inserted")


connected
success: Reset Table Neighbourhoods
success: Create Table Neighbourhoods


True

In [6]:
import numpy as np
DEFAULT_VALUE = ''

def clean(data, column_key, convert_function, default_value):
    special_values= {} 
    for row in data:
        old_value = row[column_key]
        new_value = default_value
        try:
            if old_value == '':
                new_value = 0
            else:
                new_value = convert_function(old_value)
        except (ValueError, TypeError):
            print('Replacing {} with {} in column {}'.format(row[column_key], new_value, column_key))
        row[column_key] = new_value
             

In [7]:
# data cleaning

clean(data_Neighbourhoods, 'number_of_businesses', int, '')

In [13]:
import requests
import json
def map_api(area_name):
    base_url = 'http://maps.googleapis.com/maps/api/geocode/json'
    my_params= {'address': area_name,'language':'en'}
    response = requests.get(base_url, params = my_params)

    if()
    
    results      = response.json()['results']
    nhood_geo    = results[0]['geometry']['location']
    nhood_bounds = results[0]['geometry']['bounds']
    
    #check the raw response
    
    #print(my_params['address'])
    #print("Location: ", nhood_geo['lat'], nhood_geo['lng'])
    print("Boundary: ", json.dumps(nhood_bounds, indent=4))
    return nhood_bounds

In [14]:
for row in statistic:
    boundry = map_api(row['area_name'])
    boundry_string = str(boundry['northeast']['lat']) +',' + str(boundry['northeast']['lng']) +',' + str(boundry['southwest']['lat'])+',' + str(boundry['southwest']['lng'])
    row.update({'boundry' : boundry_string})
    insert_stmt = """INSERT INTO Neighbourhoods(
                        area_id,
                         area_name,
                         land_area,
                         population,
                         number_of_dwellings,
                         number_of_businesses,
                         boundry
                    ) 
                         VALUES (%(area_id)s, %(area_name)s, %(land_area)s,%(population)s, %(number_of_dwellings)s, %(number_of_businesses)s,%(boundry)s)"""
    pgexec (conn, insert_stmt, row, "row inserted")


Boundary:  {
    "northeast": {
        "lat": -33.4794739,
        "lng": 151.4471164
    },
    "southwest": {
        "lat": -33.4933219,
        "lng": 151.4225143
    }
}
success: row inserted
Boundary:  {
    "northeast": {
        "lat": -33.4814458,
        "lng": 151.4292066
    },
    "southwest": {
        "lat": -33.5136605,
        "lng": 151.3934751
    }
}
success: row inserted
Boundary:  {
    "northeast": {
        "lat": -33.3754857,
        "lng": 151.2496735
    },
    "southwest": {
        "lat": -33.4496069,
        "lng": 151.2008984
    }
}
success: row inserted


IndexError: list index out of range